In [2]:
import logging
import os
from moviepy import VideoFileClip, ImageClip, concatenate_videoclips, vfx
import numpy as np


In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(logging.Formatter(
        '[%(levelname)s] %(asctime)s - %(name)s: %(message)s',
        datefmt='%H:%M:%S'
    ))
    logger.addHandler(console_handler)


def get_logger(name: str = __name__):
    """Trả về logger cho module khác."""
    return logging.getLogger(name)

In [4]:
def get_video_resolution(path: str) -> tuple[int, int]:
    """Trả về (width, height) của video/ảnh."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    clip = VideoFileClip(path)
    w, h = clip.size
    clip.close()
    return int(w), int(h)


In [ ]:
def trim_video(input_path: str, start: float, end: float):
    """
    Cắt video từ giây `start` đến `end`.
    """
    if end <= start:
        raise ValueError("'end' phải lớn hơn 'start'.")

    logger.info(f"Trimming {input_path} from {start}s to {end}s")
    clip = VideoFileClip(input_path).subclipped(start, end)
    return clip


def concat_videos(videos: list[str|VideoFileClip]):
    """Ghép nhiều video thành một."""
    if len(videos) < 2:
        raise ValueError("Cần ít nhất 2 video để ghép.")
    clips= []
    for v in videos:
        if isinstance(v, str):
            clips.append(VideoFileClip(v))
        elif isinstance(v, VideoFileClip):
            clips.append(v) 
        else:
            raise TypeError("Only support VideoFileCLip or str")

    logger.info(f"Concatenating {len(videos)} videos")
    final_clip = concatenate_videoclips(clips, method="chain")
    return final_clip

In [8]:
# data
import os
root = "D:/ThienPV/code/demo/data/"
img = root +"img.jpg"
docs = root + "sample_lesson.md"
video = root + "TestAPI.mp4"

In [18]:
# Test trim
start = 0
end = 4.65
short_clip = trim_video(input_path=video, start=start, end=end)
assert short_clip.duration == end -start

[INFO] 11:27:43 - __main__: Trimming D:/ThienPV/code/demo/data/TestAPI.mp4 from 0s to 4.65s


In [22]:
concated_clip = concat_videos([short_clip,short_clip,short_clip])
assert concated_clip.duration == 3 * short_clip.duration

[INFO] 11:33:07 - __main__: Concatenating 3 videos


In [52]:
from typing import Union, List, Dict, Tuple, Optional
import numpy as np
from moviepy import VideoFileClip
from PIL import Image

def top_colors_first_frame(
    video: Union[str, "VideoFileClip"],
    top_k: int = 10,
    quantize: int = 0,
    resize_to: Optional[int] = None,
    return_hex: bool = True,
) -> List[Dict]:
    """
    Lấy frame đầu (t=0) và trả về top_k màu xuất hiện nhiều nhất.

    Params
    ------
    video       : đường dẫn hoặc VideoFileClip (MoviePy v2).
    top_k       : số màu cần lấy (mặc định 10).
    quantize    : bước lượng tử hoá kênh màu (0 = đếm màu chính xác).
                  Ví dụ 16 -> gom màu theo bậc 16 (giảm nhiễu).
    resize_to   : nếu set (vd 720), sẽ downscale cạnh dài của frame
                  về <= giá trị này để tăng tốc đếm (không bắt buộc).
    return_hex  : có trả kèm mã HEX hay không.

    Returns
    -------
    List[Dict] với mỗi phần tử:
      {
        "rgb": (r, g, b),
        "hex": "#RRGGBB",   # nếu return_hex=True
        "count": <số pixel>,
        "ratio": <tỷ lệ pixel trên toàn frame>
      }
    """
    opened_here = False
    clip = video
    if isinstance(video, str):
        clip = VideoFileClip(video)
        opened_here = True
    try:
        frame = clip.get_frame(0.0)  # (H,W,3|4)
    finally:
        if opened_here:
            clip.close()

    # Chuẩn hoá về RGB uint8
    if frame.ndim != 3 or frame.shape[2] < 3:
        raise ValueError("Frame không hợp lệ (cần dạng HxWx3 hoặc HxWx4).")
    frame = frame[:, :, :3]
    if frame.dtype != np.uint8:
        # MoviePy đôi khi trả float [0..1]; chuẩn hoá về uint8
        if frame.max() <= 1.0:
            frame = np.clip(np.rint(frame * 255.0), 0, 255).astype(np.uint8)
        else:
            frame = np.clip(np.rint(frame), 0, 255).astype(np.uint8)

    # Tùy chọn downscale để tăng tốc
    if resize_to:
        h, w = frame.shape[:2]
        scale = resize_to / max(h, w) if max(h, w) > resize_to else 1.0
        if scale < 1.0:
            new_size = (int(w * scale), int(h * scale))
            frame = np.array(Image.fromarray(frame).resize(new_size, Image.BILINEAR), dtype=np.uint8)

    # Tùy chọn lượng tử hoá màu (gom cụm màu gần nhau)
    if quantize and quantize > 1:
        q = int(quantize)
        frame = (frame.astype(np.int16) // q) * q + q // 2
        frame = np.clip(frame, 0, 255).astype(np.uint8)

    # Đếm nhanh bằng numpy.unique trên dtype cấu trúc
    pixels = frame.reshape(-1, 3)
    dtype = np.dtype([("r", np.uint8), ("g", np.uint8), ("b", np.uint8)])
    structured = pixels.view(dtype)
    uniques, counts = np.unique(structured, return_counts=True)

    if counts.size == 0:
        return []

    # Lấy top_k bằng argpartition (O(n))
    k = min(top_k, counts.size)
    idx = np.argpartition(-counts, kth=k - 1)[:k]
    idx = idx[np.argsort(-counts[idx])]  # sắp xếp lại theo count giảm dần

    total = pixels.shape[0]
    results: List[Dict] = []
    for i in idx:
        r, g, b = int(uniques[i]["r"]), int(uniques[i]["g"]), int(uniques[i]["b"])
        cnt = int(counts[i])
        item: Dict[str, object] = {"rgb": (r, g, b), "count": cnt, "ratio": cnt / total}
        if return_hex:
            item["hex"] = f"#{r:02X}{g:02X}{b:02X}"
        results.append(item)

    return results


In [57]:
background_color = top_colors_first_frame(video = short_clip, top_k=2)

In [79]:
from moviepy.video.fx.MaskColor import MaskColor

no_bg_clip = MaskColor(color = background_color[0]["rgb"], threshold= 20, stiffness=1).apply(short_clip)

In [80]:
no_bg_clip.save_frame(filename="temp.png",t=1)

In [ ]:

def overlay_background(fg_path: str, bg_path: str, output_path: str,
                       position: tuple[int, int] = (0, 0), scale: float = 1.0,
                       bg_loop: bool = True, duration: float | None = None):
    """
    Overlay video foreground (đã xóa nền) lên background.
    """
    logger.info(f"Overlaying {fg_path} on {bg_path} → {output_path}")

    # Load background
    if bg_path.lower().endswith((".jpg", ".png", ".jpeg", ".webp")):
        bg_clip = ImageClip(bg_path)
        if duration:
            bg_clip = bg_clip.with_duration(duration)
        elif bg_loop:
            temp_fg = VideoFileClip(fg_path)
            bg_clip = bg_clip.with_duration(temp_fg.duration)
            temp_fg.close()
    else:
        bg_clip = VideoFileClip(bg_path)
        if duration:
            bg_clip = bg_clip.subclip(0, duration)

    # Load foreground
    fg_clip = VideoFileClip(fg_path)
    if scale != 1.0:
        fg_clip = fg_clip.resize(scale)

    # Overlay
    final = bg_clip.overlay(fg_clip, position=position)
    return final

In [84]:
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip
def overlay(fg, bg,
            position: tuple[int, int] = (0, 0), relative:bool= False, scale: float = 1.0,
            bg_loop: bool = True, duration: float | None = None):
    """
    Overlay video foreground (đã xóa nền) lên background.
    """
    # Load foreground
    if isinstance(fg,str):
        fg_clip = VideoFileClip(fg)
    else:
        fg_clip = fg
    # Load background
    if isinstance(bg, str):
        if bg.lower().endswith((".jpg", ".png", ".jpeg", ".webp")):
            bg_clip = ImageClip(bg)
            if duration:
                bg_clip = bg_clip.with_duration(duration)
            elif bg_loop:
                bg_clip = bg_clip.with_duration(fg_clip.duration)

        else:
            bg_clip = VideoFileClip(bg)
            if duration:
                bg_clip = bg_clip.subclip(0, duration)
    else:
        bg_clip = bg

    if scale != 1.0:
        fg_clip = fg_clip.resized(scale)

    # Overlay
    final = CompositeVideoClip([bg_clip,fg_clip.with_position(position, relative = relative)])
    return final

In [81]:
img

'D:/ThienPV/code/demo/data/img.jpg'

In [88]:
overlayed = overlay(fg = no_bg_clip, bg=img,scale=0.1, position=(0,0), relative=False)

In [89]:
overlayed.preview()